In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/Users/lcamacho/Triple Ten/Sprint 10/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


#### Lectura de datos

Los datos se leyeron usando la función `pd.read_csv('/datasets/users_behavior.csv')` de la librería `pandas`. Se verificó la información general con `df.info()` y las primeras filas con `df.head()`, confirmando que no había valores nulos y que los tipos de datos eran apropiados (variables de entrada `float64`, objetivo `int64`). También se verificó y confirmó la ausencia de filas duplicadas.


In [5]:

duplicated_rows = df.duplicated().sum()
print(f"Número de filas duplicadas encontradas: {duplicated_rows}")


Número de filas duplicadas encontradas: 0


In [6]:
X = df.drop('is_ultra', axis=1)
y = df['is_ultra'] 

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=12345, stratify=y
)

In [8]:
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=12345, stratify=y_temp
)

In [9]:
print("\n--- Tamaños de los Conjuntos de Datos ---")
print(f"Total de Filas: {df.shape[0]}")
print(f"Conjunto de Entrenamiento (60%): {X_train.shape[0]} filas | Etiquetas: {y_train.shape[0]}")
print(f"Conjunto de Validación (20%): {X_valid.shape[0]} filas | Etiquetas: {y_valid.shape[0]}")
print(f"Conjunto de Prueba (20%): {X_test.shape[0]} filas | Etiquetas: {y_test.shape[0]}")
print(f"Suma de Filas: {X_train.shape[0] + X_valid.shape[0] + X_test.shape[0]}")


--- Tamaños de los Conjuntos de Datos ---
Total de Filas: 3214
Conjunto de Entrenamiento (60%): 1928 filas | Etiquetas: 1928
Conjunto de Validación (20%): 643 filas | Etiquetas: 643
Conjunto de Prueba (20%): 643 filas | Etiquetas: 643
Suma de Filas: 3214


**Los datos se segmentaron correctamente, se hicieron en tres conjuntos distintos**
* Entrenmiento, X_trainy_train para poder enseñar el modelo
* Validacion, X_validy_valid para seleccionar los hiperparametros y seleccionar el mejor modelo
* Prueba, X_testy_test para evaluar la exactitud del modelo

Se utilizo un argumento en ambas divisiones. Esto asegura que la proporcion de la clase objetivo sea, se mantengan similares
**Separacion correcta** se garantizo que entre los conjuntos, los datos utilizados para entrenar no se usen para validar y que los datos de validacion no se usen para la prueba final, esto evita que exista un sobreajuste y los resultados sean muy realistas 

**Tamaño de los conjuntos**
El tamaño de los conjuntos fue de acuerdo el estandard comun y recomendado, enfoncandome que exsita un equibrio entre la cantidad de datos para el entreamiento, la validacion y la prueba 
**La divisicion elegida fue 60% entrenamiento, 20% validacion, 20% prueba**

El entrenamiento es la mayor parte porque necesita una gran cantidad de datos para poder aprender los patrones y las relaciones entre las caracteristicas y el plan ULTRA

La validacion es suficiente

La prueba es similiar al conjunto de validacion, garantiza la evaluacion final con una muestra considerable, sin que afecte al resto del modelo

La alternativa comun es que sea 75% comun y 25% prueba, pero esto solo permite una prueba final, por esta razon es mas adecuada para el flujo de trabajo Entrenamiento-validacion- prueba, se usuaron hiperparametros.




**Modelo 1: Árbol de Decisión**

In [10]:
best_dt_model = None
best_dt_accuracy = 0
best_dt_depth = 0

for depth in range(1, 11): 
     
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(X_train, y_train)


    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)

    print(f"Profundidad (max_depth): {depth} | Exactitud en Validación: {accuracy:.4f}")

    
    if accuracy > best_dt_accuracy:
        best_dt_accuracy = accuracy
        best_dt_model = model
        best_dt_depth = depth

print(f"\nMEJOR Árbol de Decisión | Exactitud: {best_dt_accuracy:.4f} | Profundidad Óptima: {best_dt_depth}")

Profundidad (max_depth): 1 | Exactitud en Validación: 0.7403
Profundidad (max_depth): 2 | Exactitud en Validación: 0.7729
Profundidad (max_depth): 3 | Exactitud en Validación: 0.7776
Profundidad (max_depth): 4 | Exactitud en Validación: 0.7543
Profundidad (max_depth): 5 | Exactitud en Validación: 0.7854
Profundidad (max_depth): 6 | Exactitud en Validación: 0.7745
Profundidad (max_depth): 7 | Exactitud en Validación: 0.7869
Profundidad (max_depth): 8 | Exactitud en Validación: 0.8025
Profundidad (max_depth): 9 | Exactitud en Validación: 0.7823
Profundidad (max_depth): 10 | Exactitud en Validación: 0.7729

MEJOR Árbol de Decisión | Exactitud: 0.8025 | Profundidad Óptima: 8


**Modelo 2: Bosque Aleatorio**

In [11]:
best_rf_model = None
best_rf_accuracy = 0
best_rf_n_estimators = 0
best_rf_depth = 0

for n_est in [10, 50, 100]: 
    for depth in range(1, 11): 
        
        model = RandomForestClassifier(
            random_state=12345,
            n_estimators=n_est,
            max_depth=depth
        )
        model.fit(X_train, y_train)

        
        predictions = model.predict(X_valid)
        accuracy = accuracy_score(y_valid, predictions)

        
        if accuracy > best_rf_accuracy:
            best_rf_accuracy = accuracy
            best_rf_model = model
            best_rf_n_estimators = n_est
            best_rf_depth = depth


print(f"MEJOR Bosque Aleatorio | Exactitud: {best_rf_accuracy:.4f} | Est.: {best_rf_n_estimators} | Prof. Óptima: {best_rf_depth}")

MEJOR Bosque Aleatorio | Exactitud: 0.8180 | Est.: 50 | Prof. Óptima: 9


**Modelo 3: Regresión Logística**

In [12]:
best_lr_model = None
best_lr_accuracy = 0


for c_val in [0.01, 0.1, 1, 10]:
    
    model = LogisticRegression(random_state=12345, solver='liblinear', C=c_val, max_iter=1000)
    model.fit(X_train, y_train)

   
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)

    
    if accuracy > best_lr_accuracy:
        best_lr_accuracy = accuracy
        best_lr_model = model

print(f"MEJOR Regresión Logística | Exactitud: {best_lr_accuracy:.4f}")

MEJOR Regresión Logística | Exactitud: 0.7185


In [13]:
final_model = RandomForestClassifier(
    random_state=12345,
    n_estimators=50,
    max_depth=9
)
final_model.fit(X_train, y_train)
predictions_test = final_model.predict(X_test)
final_accuracy = accuracy_score(y_test, predictions_test)

print(f"Modelo Final Utilizado: Bosque Aleatorio")
print(f"Exactitud Final: {final_accuracy:.4f}")
print(f"Umbral Requerido: 0.75")

Modelo Final Utilizado: Bosque Aleatorio
Exactitud Final: 0.8149
Umbral Requerido: 0.75




#### Modelos de regreción

Se probaron tres modelos principales, ajustando sus hiperparámetros en el conjunto de **Validación**:

| Modelo | Hiperparámetros explorados | Mejor Rendimiento en Validación | Hiperparámetros Óptimos |
| :--- | :--- | :--- | :--- |
| **Árbol de Decisión** | `max_depth` (1 a 10) | $\mathbf{0.8025}$ | `max_depth` = 8 |
| **Bosque Aleatorio** | `n_estimators` (Ej. 10, 50, 100) y `max_depth` (1 a 10) | $\mathbf{0.8180}$ | `n_estimators` = 50, `max_depth` = 9 |
| **Regresión Logística** | `C` (parámetro de regularización) y `solver='liblinear'` | $0.7185$ | (El modelo no fue seleccionado) |

#### ¿Evaluaste correctamente la calidad del modelo? / ¿Probaste los modelos correctamente?

**Sí, la evaluación fue correcta.**

1.  **Ajuste:** La calidad de los modelos se evaluó inicialmente en el conjunto de **Validación** (20%) para seleccionar el mejor modelo y sus hiperparámetros óptimos. Esto evitó el sobreajuste.
2.  **Prueba Final:** El modelo con el mejor desempeño en validación (**Bosque Aleatorio**) se probó una única vez en el conjunto de **Prueba** (20%) para obtener una métrica de exactitud final imparcial.

#### ¿Cuáles fueron tus hallazgos?

1.  **Superioridad del Bosque Aleatorio:** El modelo **Bosque Aleatorio** demostró ser el más efectivo, logrando una exactitud en validación de $0.8180$.
2.  **Naturaleza No Lineal del Problema:** Los modelos no lineales (Árbol de Decisión y Bosque Aleatorio) superaron significativamente al modelo lineal (Regresión Logística, con $0.7185$). Esto sugiere que la relación entre los datos de comportamiento del cliente y la elección del plan no es una simple línea recta, sino que requiere límites de decisión más complejos.
3.  **Generalización Exitosa:** El modelo final (Bosque Aleatorio) mantuvo una exactitud alta al pasar del conjunto de validación ($0.8180$) al conjunto de prueba ($\mathbf{0.8149}$), indicando que el modelo generaliza bien y no está sobreajustado.

#### ¿Cuál es tu puntuación de exactitud?

La puntuación de exactitud final, obtenida en el conjunto de prueba con el modelo Bosque Aleatorio optimizado, es de **0.8149**.

$$\text{Puntuación de Exactitud Final} = \mathbf{0.8149}$$

*   **Umbral Mínimo Requerido:** 0.75
*   **Estado:** **Éxito**.

#### ¿Te ceñiste a la estructura del proyecto y mantuviste limpio el código?

**Sí.** El proyecto siguió la estructura estándar de *machine learning* para clasificación:

1.  Carga y examen de datos.
2.  Segmentación rigurosa en conjuntos de Entrenamiento/Validación/Prueba (60/20/20).
3.  Entrenamiento y ajuste de múltiples modelos con hiperparámetros.
4.  Selección del modelo campeón.
5.  Prueba final en un conjunto de datos nunca antes visto.

El código utilizó librerías estándar de `pandas` y `sklearn` de manera clara y organizada.